In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


## Package

In [2]:
!pip install pymysql
!sudo apt-get install -y fonts-nanum # 나눔 폰트 설치 (설치 후 런타임 재시작하고 코드 실행)
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43 kB 1.5 MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 20 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-nanum all 20170925-1 [9,604 kB]
Fetched 9,604 kB in 3s (3,091 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend

In [3]:
import numpy as np
import pandas as pd
import pymysql
from sqlalchemy import create_engine
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from matplotlib import font_manager, rc
import random
font_name = fm.FontProperties().get_name()
plt.rc('font', family=font_name) # 한글 출력

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Data load

In [5]:
# DB 연결
endpoint = 'dao.c51deksujiip.ap-northeast-2.rds.amazonaws.com'
schema_name = 'dao'
db_connection_str = 'mysql+pymysql://admin:ekfkawnl@{}/{}'.format(endpoint, schema_name )
try :
    db_connection = create_engine(db_connection_str)
    conn = db_connection.connect()
except :
    print('fail to connect db')

In [6]:
def sql_to_df(table_name):
    sql = "SELECT * FROM {}".format(table_name)
    df = pd.read_sql(sql, db_connection)
    return df

In [7]:
df = sql_to_df('MainData_식생')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 497 entries, 0 to 496
Data columns (total 30 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   번호                497 non-null    int64  
 1   위도                497 non-null    float64
 2   경도                497 non-null    float64
 3   생태통로_유형           497 non-null    int64  
 4   등산객_이용빈도          497 non-null    int64  
 5   생태통로_효율성          497 non-null    int64  
 6   이용확인종_수           497 non-null    int64  
 7   폭(m)              309 non-null    float64
 8   연장(m)             309 non-null    float64
 9   높이(m)             44 non-null     float64
 10  개방도               44 non-null     float64
 11  경사도               497 non-null    object 
 12  주변 로드킬 빈도         497 non-null    int64  
 13  산책로까지의 최단 거리(km)  497 non-null    float64
 14  농가까지의 거리(km)      497 non-null    float64
 15  도로 최고제한속도(km/h)   497 non-null    int64  
 16  주변 동물종 개수         497 non-null    int64  
 1

## Labeling

지형기호(2.3km)

In [8]:
df['topography'] = [x.split('-')[0] for x in df['topography']]
df['topography'].unique()

array(['X', '하천', '산지', '기타', '해안', '화산'], dtype=object)

생태통로 효율성 라벨링 0(낮음)/1(일정 수준 이상)

In [9]:
df['efficiency'] = [1 if x==2 else x for x in df['efficiency']]

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 497 entries, 0 to 496
Data columns (total 30 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   번호                497 non-null    int64  
 1   위도                497 non-null    float64
 2   경도                497 non-null    float64
 3   생태통로_유형           497 non-null    int64  
 4   등산객_이용빈도          497 non-null    int64  
 5   생태통로_효율성          497 non-null    int64  
 6   이용확인종_수           497 non-null    int64  
 7   폭(m)              309 non-null    float64
 8   연장(m)             309 non-null    float64
 9   높이(m)             44 non-null     float64
 10  개방도               44 non-null     float64
 11  경사도               497 non-null    object 
 12  주변 로드킬 빈도         497 non-null    int64  
 13  산책로까지의 최단 거리(km)  497 non-null    float64
 14  농가까지의 거리(km)      497 non-null    float64
 15  도로 최고제한속도(km/h)   497 non-null    int64  
 16  주변 동물종 개수         497 non-null    int64  
 1

## 육교형/터널형 Split

In [11]:
df_overpass = df[df['type'] == 0] # 육교형
df_tunnel = df[df['type'] != 0] # 터널형

# 육교형에서 사용하지 않는 열 삭제
df_overpass.drop(['opennesss', 'height'], axis=1, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [12]:
# index 초기화
df_overpass.reset_index(drop = True,inplace= True)
df_tunnel.reset_index(drop = True,inplace= True)

In [13]:
df_overpass.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 28 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   번호                300 non-null    int64  
 1   위도                300 non-null    float64
 2   경도                300 non-null    float64
 3   생태통로_유형           300 non-null    int64  
 4   등산객_이용빈도          300 non-null    int64  
 5   생태통로_효율성          300 non-null    int64  
 6   이용확인종_수           300 non-null    int64  
 7   폭(m)              264 non-null    float64
 8   연장(m)             264 non-null    float64
 9   경사도               300 non-null    object 
 10  주변 로드킬 빈도         300 non-null    int64  
 11  산책로까지의 최단 거리(km)  300 non-null    float64
 12  농가까지의 거리(km)      300 non-null    float64
 13  도로 최고제한속도(km/h)   300 non-null    int64  
 14  주변 동물종 개수         300 non-null    int64  
 15  주변동물 출현빈도         300 non-null    int64  
 16  지형기호(2.3km)       300 non-null    object 
 1

In [14]:
df_tunnel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197 entries, 0 to 196
Data columns (total 30 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   번호                197 non-null    int64  
 1   위도                197 non-null    float64
 2   경도                197 non-null    float64
 3   생태통로_유형           197 non-null    int64  
 4   등산객_이용빈도          197 non-null    int64  
 5   생태통로_효율성          197 non-null    int64  
 6   이용확인종_수           197 non-null    int64  
 7   폭(m)              45 non-null     float64
 8   연장(m)             45 non-null     float64
 9   높이(m)             43 non-null     float64
 10  개방도               43 non-null     float64
 11  경사도               197 non-null    object 
 12  주변 로드킬 빈도         197 non-null    int64  
 13  산책로까지의 최단 거리(km)  197 non-null    float64
 14  농가까지의 거리(km)      197 non-null    float64
 15  도로 최고제한속도(km/h)   197 non-null    int64  
 16  주변 동물종 개수         197 non-null    int64  
 1

## 이상치

### 이상치 확인

In [15]:
# 이상치 처리해야 할 컬럼 목록
overpass_check = ['num_lane', 'num_used_species', 'width', 'extension', 'dist_walkway',
                  'dist_farm', 'speed_limit', 'num_species', 'frequency_animals', 'dist_river', 
                  '등산로까지 최단거리(km)', '유도울타리_연장_m', '유도울타리_높이_m', '교통량', '건물까지거리(km)']

tunnel_check = ['num_lane', 'num_used_species', 'width', 'extension', 'height', 'openness',
                '산책로까지의 최단 거리(km)', '농가까지의 거리(km)', '도로 최고제한속도(km/h)', '주변 동물종 개수', '주변동물 출현빈도', 
                '하천거리(km)', '등산로까지 최단거리(km)', '유도울타리_연장_m', '유도울타리_높이_m', '교통량', '건물까지거리(km)']

print(len(overpass_check))
print(len(tunnel_check))

15
17


In [16]:
# # 육교형 생태통로 Boxplot
# plt.figure(figsize=(16, 10))

# for i, col in enumerate(overpass_check):
#   plt.subplot(3, 6, i+1)
#   df_overpass.boxplot(col)

# plt.tight_layout()
# plt.show()

In [17]:
# # 터널형 생태통로 Boxplot
# plt.figure(figsize=(16, 10))

# for i, col in enumerate(tunnel_check):
#   plt.subplot(4, 5, i+1)
#   df_tunnel.boxplot(col)

# plt.tight_layout()
# plt.show()

### 이상치 처리

tukey fence 방식으로 처리

In [18]:
# 높은 이상치는 Boxplot의 상한값으로, 낮은 이상치는 Boxplot의 하한값으로 대체하는 함수
def change_outlier(df, col_list):
  for col_name in col_list:
      Q1 = df[col_name].quantile(q=0.25)
      Q3 = df[col_name].quantile(q=0.75)
      IQR = Q3-Q1

      print('[' + col_name + ']', end=' ')
      high_idx = df[df[col_name] > (Q3+1.5*IQR)].index
      df.loc[high_idx, col_name] = round(Q3+1.5*IQR, 1)
      print(str(len(high_idx)) +'개가 상한값으로 변경됨', end=' / ')

      low_idx = df[df[col_name] < (Q1-1.5*IQR)].index
      df.loc[low_idx, col_name] = round(Q1-1.5*IQR, 1)
      print(str(len(low_idx)) +'개가 하한값으로 변경됨')

  return df

In [19]:
df_overpass = change_outlier(df_overpass, overpass_check)
df_tunnel = change_outlier(df_tunnel, tunnel_check)

[차선_수] 8개가 상한값으로 변경됨 / 0개가 하한값으로 변경됨
[이용확인종_수] 2개가 상한값으로 변경됨 / 0개가 하한값으로 변경됨
[폭(m)] 21개가 상한값으로 변경됨 / 0개가 하한값으로 변경됨
[연장(m)] 9개가 상한값으로 변경됨 / 0개가 하한값으로 변경됨
[산책로까지의 최단 거리(km)] 0개가 상한값으로 변경됨 / 0개가 하한값으로 변경됨
[농가까지의 거리(km)] 26개가 상한값으로 변경됨 / 0개가 하한값으로 변경됨
[도로 최고제한속도(km/h)] 0개가 상한값으로 변경됨 / 0개가 하한값으로 변경됨
[주변 동물종 개수] 3개가 상한값으로 변경됨 / 0개가 하한값으로 변경됨
[주변동물 출현빈도] 19개가 상한값으로 변경됨 / 

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


0개가 하한값으로 변경됨
[하천거리(km)] 15개가 상한값으로 변경됨 / 0개가 하한값으로 변경됨
[등산로까지 최단거리(km)] 16개가 상한값으로 변경됨 / 0개가 하한값으로 변경됨
[유도울타리_연장_m] 59개가 상한값으로 변경됨 / 0개가 하한값으로 변경됨
[유도울타리_높이_m] 0개가 상한값으로 변경됨 / 0개가 하한값으로 변경됨
[교통량] 28개가 상한값으로 변경됨 / 0개가 하한값으로 변경됨
[건물까지거리(km)] 23개가 상한값으로 변경됨 / 0개가 하한값으로 변경됨
[차선_수] 5개가 상한값으로 변경됨 / 0개가 하한값으로 변경됨
[이용확인종_수] 8개가 상한값으로 변경됨 / 0개가 하한값으로 변경됨
[폭(m)] 2개가 상한값으로 변경됨 / 0개가 하한값으로 변경됨
[연장(m)] 2개가 상한값으로 변경됨 / 0개가 하한값으로 변경됨
[높이(m)] 0개가 상한값으로 변경됨 / 0개가 하한값으로 변경됨
[개방도] 1개가 상한값으로 변경됨 / 0개가 하한값으로 변경됨
[산책로까지의 최단 거리(km)] 0개가 상한값으로 변경됨 / 0개가 하한값으로 변경됨
[농가까지의 거리(km)] 27개가 상한값으로 변경됨 / 0개가 하한값으로 변경됨
[도로 최고제한속도(km/h)] 0개가 상한값으로 변경됨 / 0개가 하한값으로 변경됨
[주변 동물종 개수] 5개가 상한값으로 변경됨 / 0개가 하한값으로 변경됨
[주변동물 출현빈도] 16개가 상한값으로 변경됨 / 0개가 하한값으로 변경됨
[하천거리(km)] 7개가 상한값으로 변경됨 / 0개가 하한값으로 변경됨
[등산로까지 최단거리(km)] 12개가 상한값으로 변경됨 / 0개가 하한값으로 변경됨
[유도울타리_연장_m] 18개가 상한값으로 변경됨 / 0개가 하한값으로 변경됨
[유도울타리_높이_m] 0개가 상한값으로 변경됨 / 0개가 하한값으로 변경됨
[교통량] 13개가 상한값으로 변경됨 / 0개가 하한값으로 변경됨
[건물까지거리(km)] 14개가 상한값으로 변경됨 / 0개가 하한값으로 변경됨


In [20]:
# 이상치 처리된 MainData DF → DB로 업로드
# df_overpass.to_sql(name='MainData_outlier_overpass', con=db_connection, if_exists='replace', index=False)
# df_tunnel.to_sql(name='MainData_outlier_tunnel', con=db_connection, if_exists='replace', index=False)

## Standard Scaling, One-Hot Encoding

In [21]:
# drop, scaling, one-hot encoding, labeling 함수
def preprocess_df(df, onehot_col=None, scaling_col=None, drop_col = None, labeling_col = None, scaling_func=MinMaxScaler) :
    """
    :param df: 사용할 데이터프레임
    :param onehot_col: default = None, 원핫인코딩할 컬럼 리스트
    :param scaling_col: default = None, 스케일링할 컬럼 리스트
    :param drop_col: default = None, 사용하지 않을 컬럼 리스트
    :param labeling_col: default = None, 라벨링이 필요한 컬럼 리스트
    :param scaling_func: 스케일링시 사용할 컬럼, MinMaxScaler or StandardScaler
    :return: 전처리 된 데이터 프레임
    """
    if drop_col is not None :
        df = df.drop(drop_col, axis=1)
    if onehot_col is not None:
        df = pd.get_dummies(df, colums=onehot_col)
    if scaling_col is not None :
        for c in scaling_col :
            scaler = scaling_func()
            scaler.fit(df[[c]])
            df[c] = scaler.transform(df[[c]])
    if labeling_col is not None :
        for c in labeling_col :
            encoder = LabelEncoder()
            encoder.fit(df[c])
            df[c] = encoder.transform(df[c])
    return df


In [22]:
# column list
onehot_col = ['등산객_이용빈도', '경사도', '지형기호(2.3km)', '환경영향평가점수']
scaling_col = ['이용확인종_수', 'width', '연장(m)', '주변 로드킬 빈도', '산책로까지의 최단 거리(km)', '농가까지의 거리(km)', '도로 최고제한속도(km/h)',
               '하천거리(km)', '등산로까지 최단거리(km)', '건물까지거리(km)', '교통량', '주변 동물종 개수', '주변동물 출현빈도', '유도울타리_연장_m', '유도울타리_높이_m',]
drop_col = ['번호', '위도', '경도', '생태통로_유형', '유도울타리_위도_2.3km', '유도울타리_경도_2.3km']
scaling_col_tunnel = scaling_col + ['높이(m)', '개방도']

In [23]:
df_overpass = preprocess_df(df_overpass, drop_col=drop_col, labeling_col=onehot_col, scaling_col=scaling_col, scaling_func=StandardScaler)
df_tunnel = preprocess_df(df_tunnel, drop_col=drop_col, labeling_col=onehot_col, scaling_col=scaling_col, scaling_func=StandardScaler)

## 결측치

In [24]:
# 유도울타리 높이, 연장, 주변동물 출현빈도, 주변 동물종 개수 => 존재하지 않는 것이므로 0으로 처리
df_overpass[['fence_height', 'fence_extencsion', '주변동물 출현빈도', '주변 동물종 개수']] = df_overpass[['fence_height', 'fence_extencsion', '주변동물 출현빈도', '주변 동물종 개수']].fillna(0)
df_tunnel[['fence_height', 'fence_extencsion', '주변동물 출현빈도', '주변 동물종 개수']] = df_tunnel[['fence_height', 'fence_extencsion', '주변동물 출현빈도', '주변 동물종 개수']].fillna(0)

### 규격(폭, 연장, 높이, 개방도)
Case 1. 평균으로 대체

In [25]:
# 규격 데이터의 결측치를 평균으로 대체하는 함수
def MeanImputation(df, col_list):
  df_copy = df.copy()
  for col_name in col_list:
    df_copy[col_name].fillna(np.round(df_copy[col_name].mean(), 1), inplace=True)
  return df_copy

Case 2. KNN으로 대체

In [26]:
# KNN 알고리즘으로 결측값을 대체 함수
def KNNImputation(df, n_neighbors):
  df_copy = df.copy()
  col = df_copy.columns
  imputer = KNNImputer(n_neighbors=n_neighbors)
  df_imputed = imputer.fit_transform(df_copy)
  df_copy = pd.DataFrame(df_imputed)
  return df_copy, col

In [27]:
# [Case 1] 평균으로 대체
df_overpass_mean = MeanImputation(df_overpass, ['width', '연장(m)', '교통량'])
df_tunnel_mean = MeanImputation(df_tunnel, ['width', '연장(m)', '높이(m)', '개방도', '교통량'])

In [28]:
# [Case 2] KNN으로 대체

df_over, col_over = KNNImputation(df_overpass, 3)
df_over.columns = col_over
globals()['df_overpass_knn3'] = df_over

df_tun, col_tun = KNNImputation(df_tunnel, 3)
df_tun.columns = col_tun
globals()['df_tunnel_knn3'] = df_tun

## 생태통로 효율성 '판단불가' 대체

Case 1. '판단불가' 값이 있는 행 삭제

In [29]:
# 생태통로 효율성 데이터가 '판단불가' 값인 행을 삭제하는 함수
def drop_value(df) :
    df_copy = df.copy() # 파라미터로 넣은 데이터프레임 원본이 바뀌지 않도록 복사
    idx = list(df_copy[df_copy['생태통로_효율성'] == 3].index) # 효율성 3 : 판단불가
    df_copy.drop(idx, inplace=True)
    return df_copy

In [30]:
# 생태통로 효율성 데이터가 '판단불가' 값이 아닌 행을 삭제하는 함수
def drop_value_not3(df) :
    df_copy = df.copy() # 파라미터로 넣은 데이터프레임 원본이 바뀌지 않도록 복사
    idx = list(df_copy[df_copy['생태통로_효율성'] != 3].index) # 효율성 3 : 판단불가
    df_copy.drop(idx, inplace=True)
    return df_copy

Case 2. '판단불가' 값을 중앙값('보통')으로 대체

In [31]:
# 생태통로 효율성 데이터가 '판단불가' 값이면 중앙값('보통')으로 변경하는 함수
def change_median(df):
  df_copy=df.copy()
  df_copy['생태통로_효율성']=[1 if x==3 else x for x in df_copy['생태통로_효율성']]
  return df_copy

In [32]:
# 결측치 : 평균 / 효율성 : '판단불가' 값 삭제 & '판단불가' 값 제외 삭제

overpass_mean_drop = drop_value(df_overpass_mean)
overpass_mean_drop_not3 = drop_value_not3(df_overpass_mean)

tunnel_mean_drop = drop_value(df_tunnel_mean)
tunnel_mean_drop_not3 = drop_value_not3(df_tunnel_mean)

In [33]:
# 결측치 : KNN / 효율성 : '판단불가' 값 삭제 & '판단불가' 값 제외 삭제

overpass_knn3_drop = drop_value(df_overpass_knn3)
overpass_knn3_drop_not3 = drop_value_not3(df_overpass_knn3)

tunnel_knn3_drop = drop_value(df_tunnel_knn3)
tunnel_knn3_drop_not3 = drop_value_not3(df_tunnel_knn3)

In [34]:
# 결측치 : 평균 / 효율성 : 중앙값('보통')

overpass_mean_median = change_median(df_overpass_mean)
tunnel_mean_median = change_median(df_tunnel_mean)

In [35]:
# 결측치 : 평균 / 효율성 : 중앙값('보통')

overpass_knn3_median = change_median(df_overpass_knn3)
tunnel_knn3_median = change_median(df_tunnel_knn3)

## Final Data

In [36]:
df_list = [overpass_mean_drop, overpass_mean_drop_not3, tunnel_mean_drop, tunnel_mean_drop_not3, overpass_mean_median, tunnel_mean_median, 
           overpass_knn3_drop, overpass_knn3_drop_not3, tunnel_knn3_drop, tunnel_knn3_drop_not3, overpass_knn3_median, tunnel_knn3_median]        
print(len(df_list))
print([df.shape for df in df_list])

12
[(183, 22), (117, 22), (122, 24), (75, 24), (300, 22), (197, 24), (183, 22), (117, 22), (122, 24), (75, 24), (300, 22), (197, 24)]


In [38]:
# DF to CSV
path = '/content/drive/MyDrive/데이터청년캠퍼스/Team2_Data File/maindata/'

overpass_mean_drop.to_csv(path + 'overpass_mean_drop.csv', index=False, encoding='cp949')
tunnel_mean_drop.to_csv(path + 'tunnel_mean_drop.csv', index=False, encoding='cp949')
overpass_mean_drop_not3.to_csv(path + 'overpass_mean_drop_not3.csv', index=False, encoding='cp949')
tunnel_mean_drop_not3.to_csv(path + 'tunnel_mean_drop_not3.csv', index=False, encoding='cp949')

overpass_mean_median.to_csv(path + 'overpass_mean_median.csv', index=False, encoding='cp949')
tunnel_mean_median.to_csv(path + 'tunnel_mean_median.csv', index=False, encoding='cp949')

overpass_knn3_drop.to_csv(path + 'overpass_knn3_drop.csv', index=False, encoding='cp949')
tunnel_knn3_drop.to_csv(path + 'tunnel_knn3_drop.csv', index=False, encoding='cp949')
overpass_knn3_drop_not3.to_csv(path + 'overpass_knn3_drop_not3.csv', index=False, encoding='cp949')
tunnel_knn3_drop_not3.to_csv(path + 'tunnel_knn3_drop_not3.csv', index=False, encoding='cp949')

overpass_knn3_median.to_csv(path + 'overpass_knn3_median.csv', index=False, encoding='cp949')
tunnel_knn3_median.to_csv(path + 'tunnel_knn3_median.csv', index=False, encoding='cp949')